## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-01-01-16-00 +0000,wsj,South Korea’s Exports Stronger Than Expected i...,https://www.wsj.com/economy/trade/south-koreas...
1,2026-01-01-01-12-14 +0000,nyt,New Year’s 2026 Celebrations Around the World:...,https://www.nytimes.com/2025/12/31/world/new-y...
2,2026-01-01-01-09-45 +0000,startribune,Takeaways: Sharks thwart Wild 4-3 in a shootou...,https://www.startribune.com/takeaways-sharks-t...
3,2026-01-01-01-08-48 +0000,nyt,Oil Tanker Fleeing the Coast Guard Now Listed ...,https://www.nytimes.com/2025/12/31/us/politics...
4,2026-01-01-00-57-08 +0000,bbc,In pictures: New Year welcomed around the world,https://www.bbc.com/news/articles/c050j896q2do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,year,36
53,trump,35
8,new,30
26,guard,14
317,china,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
74,2025-12-31-21-46-14 +0000,nypost,Nasty weather on tap for New Year fetes at Ros...,https://nypost.com/2025/12/31/us-news/nasty-we...,124
75,2025-12-31-21-46-01 +0000,nypost,Eric Adams ends term as NYC mayor with Times S...,https://nypost.com/2025/12/31/us-news/eric-ada...,108
179,2025-12-31-11-05-20 +0000,nypost,Washington Monument will glow on New Year’s Ev...,https://nypost.com/2025/12/31/us-news/washingt...,98
106,2025-12-31-19-02-59 +0000,nypost,NJ drivers hit with 9% state gas tax hike to k...,https://nypost.com/2025/12/31/us-news/nj-drive...,95
45,2025-12-31-23-02-41 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
74,124,2025-12-31-21-46-14 +0000,nypost,Nasty weather on tap for New Year fetes at Ros...,https://nypost.com/2025/12/31/us-news/nasty-we...
81,86,2025-12-31-21-20-43 +0000,nypost,"Trump pulling National Guard from Chicago, Por...",https://nypost.com/2025/12/31/us-news/trump-re...
241,44,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...
75,42,2025-12-31-21-46-01 +0000,nypost,Eric Adams ends term as NYC mayor with Times S...,https://nypost.com/2025/12/31/us-news/eric-ada...
119,40,2025-12-31-17-56-58 +0000,nypost,OpenAI’s pay tops every major tech startup as ...,https://nypost.com/2025/12/31/business/openais...
107,35,2025-12-31-19-02-13 +0000,nypost,DOJ ‘working around the clock’ on Epstein file...,https://nypost.com/2025/12/31/us-news/doj-work...
76,33,2025-12-31-21-41-11 +0000,nypost,MTG calls for a ‘tax revolt’ after accusing Tr...,https://nypost.com/2025/12/31/us-news/marjorie...
157,33,2025-12-31-14-48-14 +0000,nypost,"Russian drone attack injures 6, including chil...",https://nypost.com/2025/12/31/world-news/russi...
18,32,2026-01-01-00-11-00 +0000,wsj,Trump’s surprise veto of a Colorado water proj...,https://www.wsj.com/politics/policy/trump-veto...
26,30,2025-12-31-23-37-55 +0000,nypost,Hero grandfather dies after running into house...,https://nypost.com/2025/12/31/us-news/grandfat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
